# Hidden symmetry 
## The motivation
* Go beyond the integeralbility 
* Exploring the new structure of non-plannar amplitude
* Understanding the gauge/gravity duality deeply 

## The expected properties of the symmetry 
* Leave the amplitude invariant 
* The hidden symmetry is a global symmetry
* There are some factors which related to the ordering in the kinematic algebra when symmetry act on the external legs 
* The structure constant of the algebra should be the BCJ numerator 
* It should be Yangian symmetry for the planar amplitude
* It have martix representation in the spinor space

# The function of the algebra product  

In [1]:
 push!(LOAD_PATH, "/Users/gangchen/Dropbox/YM_BCJ/HiddenSymmetry");

In [2]:
using vectorDot
using SymEngine
using Latexify
using LaTeXStrings

In [3]:
p1=vecSym("p_1");p2=vecSym("p_2");p3=vecSym("p_3");p4=vecSym("p_4");p5=vecSym("p_5");p6=vecSym("p_6");
p7=vecSym("p_7");
ϵ1=vecSym("ϵ_1");
ϵ2=vecSym("ϵ_2");ϵ3=vecSym("ϵ_3");ϵ4=vecSym("ϵ_4");ϵ5=vecSym("ϵ_5");ϵ6=vecSym("ϵ_6");ϵ7=vecSym("ϵ_7");
a1=symbols("a_1");
a2=symbols("a_2");
a3=symbols("a_3");

In [5]:
mutable struct J <: Number
    comp
    index::Int64 
    mom
    function J(x,i::Int64,p)
        new(x,i,p)
    end
end
 function Base.show(io::IO, cur::J)
        Base.show(io, MIME("text/latex"), cur)
 end
function Base.show(io::IO, ::MIME"text/latex", cur::J)
    id="("*string(cur.index)*")";
    mm=cur.mom;
    com=cur.comp.name;
    print(io, "\$ J^{$id}_{$com}\$","(",mm,")")
end

In [7]:
 struct scalarJTimes <: Number
        scalar::Basic
        vec::J
        function scalarJTimes(x::Basic,y::J)
            new(x,y)
        end
    end
 function Base.show(io::IO, y::scalarJTimes)
        Base.show(io, MIME("text/latex"), y)
 end
function Base.show(io::IO, ::MIME"text/latex", y::scalarJTimes)
        print(io,"[",y.scalar,"]"*" ",y.vec)
end

In [8]:
  struct JAdd <: Number
        list
        function JAdd(x)
            new(x)
        end
    end
function Base.show(io::IO, y::JAdd)
        Base.show(io, MIME("text/latex"), y)
 end
    function Base.show(io::IO,::MIME"text/latex", y::JAdd)
        str="";
        args=y.list;
        for i in 1:length(args)
            arg = args[i]
            str = string(str, arg)
            i != length(args) && (str *= " "*" "*"+"*" "*" ")
        end
        print(io, str)
    end

In [9]:
 import Base.*
    *(x::Basic,y::J)= scalarJTimes(x,y)
    *(x::Basic,y::scalarJTimes)= scalarJTimes(x*y.scalar,y.vec)
    *(y::scalarJTimes,x::Basic)=*(x::Basic,y::scalarJTimes)
    *(x::Basic,y::JAdd)= JAdd(x.*y.list)
    import Base.+
    +(x::J,y::J)= JAdd((x,y))
    #+(x::J,y...)= JAdd((x,y...))
    +(x::scalarJTimes,y::J)= JAdd((x,y))
    +(x::scalarJTimes,y::scalarJTimes)= JAdd((x,y))
   # +(x::scalarJTimes,y...)= JAdd((x,y...))
    +(x::JAdd,y::scalarJTimes)= JAdd(((x.list)...,y))
   +(y::scalarJTimes,x::JAdd)= +(x,y)
    +(x::JAdd,y::J)= JAdd(((x.list)...,y))
    +(y::J, x::JAdd)= JAdd((y, (x.list)...))
    +(y::JAdd, x::JAdd)= JAdd(((y.list)..., (x.list)...))
    #+(x::Tuple{J})= JAdd(x)

+ (generic function with 182 methods)

In [16]:
function ⋆(j1::J,j2::J)
    if j1.comp.name[1]=='ϵ'
       # print("this is epsilon",j1.comp.name[1])
        dot(j2.comp,j1.mom)*J(j1.comp,0,j1.mom+j2.mom) +1//2*dot(j1.comp,j2.comp)*J(j2.mom,0,j1.mom+j2.mom)
    elseif j1.comp.name[1]=='p'
       # print("this is p")
        dot(j2.comp,j1.mom)*J(j1.comp,0,j1.mom+j2.mom) +dot(j1.comp,j2.comp)*J(j2.mom,0,j1.mom+j2.mom)
    end
end

    function ⋆(x::scalarJTimes,y::J)
        x.scalar*⋆(x.vec,y)
    end
    function ⋆(x::JAdd,y::J)
        xx = x.list;
        s=⋆(xx[1],y);
        for i=2:length(xx)
           s+=⋆(xx[i],y)
        end
        return s
    end
    function ⋆(x::JAdd,y::scalarJTimes)
       y.scalar*⋆(x,y.vec)
    end

⋆ (generic function with 4 methods)

In [17]:
function todot(y::J)
    ynm=y.comp.name;
   return symbols(ynm*"⋅"*"ϵ_n")
end
function todot(y::scalarJTimes)
    y.scalar*todot(y.vec)
end
function todot(y::JAdd)
    yy = y.list;
        s=todot(yy[1]);
        for i=2:length(yy)
           s+=todot(yy[i])
        end
        return s
end

todot (generic function with 3 methods)

In [19]:
ttt=J((ϵ1),0,p1)⋆J((ϵ2),0,p2)⋆J((ϵ3),0,p3)

[(p_1⋅ϵ_3 + p_2⋅ϵ_3)*p_1⋅ϵ_2] $ J^{(0)}_{ϵ_1}$(+($ p_1 $, $ p_2 $, $ p_3 $))  +  [(1/2)*p_1⋅ϵ_2*ϵ_1⋅ϵ_3] $ J^{(0)}_{p_3}$(+($ p_1 $, $ p_2 $, $ p_3 $))  +  [(1/2)*(p_1⋅ϵ_3 + p_2⋅ϵ_3)*ϵ_1⋅ϵ_2] $ J^{(0)}_{p_2}$(+($ p_1 $, $ p_2 $, $ p_3 $))  +  [(1/2)*p_2⋅ϵ_3*ϵ_1⋅ϵ_2] $ J^{(0)}_{p_3}$(+($ p_1 $, $ p_2 $, $ p_3 $))

In [20]:
tt2=expand(todot(ttt))

p_1⋅ϵ_2*p_2⋅ϵ_3*ϵ_1⋅ϵ_n + (1/2)*p_1⋅ϵ_2*p_3⋅ϵ_n*ϵ_1⋅ϵ_3 + p_1⋅ϵ_3*p_1⋅ϵ_2*ϵ_1⋅ϵ_n + (1/2)*p_1⋅ϵ_3*p_2⋅ϵ_n*ϵ_1⋅ϵ_2 + (1/2)*p_2⋅ϵ_3*p_2⋅ϵ_n*ϵ_1⋅ϵ_2 + (1/2)*p_2⋅ϵ_3*p_3⋅ϵ_n*ϵ_1⋅ϵ_2

In [21]:
nice(tt2)

# BCJ relations and the map from DDM basis to minimal basis

# Ansatz of the symmetry action on the external legs

# Equations from the requirement of the amplitude invariant.